# OSM + DATA_MOS + ORS

In [ ]:
# Restart the kernel
import IPython
IPython.Application.instance().kernel.do_shutdown(True)
print('Ядро перезапущено, можно продолжать работать!')

## 1. Import libraries

In [1]:
import osmnx as ox
import geopandas as gpd
import folium
import requests
import pandas as pd

## 2. Explore osmnx library

#### 2.1 Select territory 

In [2]:
area = ox.geocode_to_gdf('Khamovniki, Moscow')

#### 2.2 Explore the boundaries we get

In [ ]:
area.head()

In [ ]:
area.plot()

#### 2.3 List key-value pairs for tags and get geometries

In [ ]:
tags = {'building': True}   

buildings = ox.geometries_from_place('Khamovniki', tags)

buildings.head()

In [ ]:
type(buildings)

#### 2.4 Select only Polygon geometries

In [ ]:
buildings = buildings.loc[buildings.geometry.type=='Polygon']
buildings.head()

#### 2.5 Plot two layers (district boundary and buildings)

In [ ]:
base = area.plot(color='white', edgecolor='blue')
buildings.plot(ax=base, color='blue');

#### 2.6 Save layers to file

2.6.1. Change columns list types to string

In [9]:
for col in buildings.columns:
    if any(isinstance(val, list) for val in buildings[col]):
        buildings = buildings.astype({col: str})


In [ ]:
buildings.to_file('building.shp')

#### 2.7 Create web-map to look at the results* (Optional)

In [ ]:
osm_map = folium.Map(location=[buildings.centroid.y.mean(), buildings.centroid.x.mean()], zoom_start=14)
polygons  = folium.features.GeoJson(buildings.to_json())
osm_map.add_child(polygons)
osm_map

## 3. DataMos API: it was never so easy to get data from data_mos ...

#### 3.1 Set parameters for the request

In [26]:
datamos_api = 'your Api Key'
data_set = 2263 # number of a dataSet (2263)

url_data = f'https://apidata.mos.ru/v1/datasets/{data_set}/features?api_key={datamos_api}'

#### 3.2 Request data

In [27]:
data_mos = requests.get(url = url_data )

In [ ]:
data_mos.status_code

In [ ]:
data_mos.json()

#### 3.3. Create GeoDataFrame

In [ ]:
schools = gpd.GeoDataFrame.from_features(data_mos.json()["features"], crs="EPSG:4326")
schools.head()

#### 3.4 Split attributes to a separate fields

In [ ]:

schools_attributes = pd.DataFrame(schools['Attributes'].values.tolist(), index=schools.index).applymap(str)
schools_final = pd.concat([schools, schools_attributes], axis = 1).drop('Attributes', axis = 1)


#### 3.5 Save data 

In [ ]:
schools_final.to_file('school.shp')

## 4. ORS_API: directions, isochrones and more 

#### 4.1 Set API

In [15]:
ors_api = '5b3ce3597851110001cf62484f4e88fde8a04bdd9dbd91f8f33b89a0'

#### 4.2 Set parameters of locations, transportation mode and isochrones range (in seconds)

In [ ]:
mode = 'foot-walking'
range = [200, 300, 400, 500]
locations = [[buildings.centroid.x.mean(), buildings.centroid.y.mean()]]


body = {"locations":locations, "range":range}
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': ors_api,
    'Content-Type': 'application/json; charset=utf-8'
}



#### 4.3 Request isochrones

In [33]:
response = requests.post(f'https://api.openrouteservice.org/v2/isochrones/{mode}', json=body, headers=headers)

#### 4.4 Explore the result

In [ ]:
response.json()

#### 4.5 Create DeoDataFrame 

In [ ]:
isochrones_gdf = gpd.GeoDataFrame.from_features(response.json()["features"], crs="EPSG:4326")
print(isochrones_gdf.head())

#### 4.6 Create web-map to look at the results

In [ ]:
osm_map = folium.Map(location=[buildings.centroid.y.mean(), buildings.centroid.x.mean()], zoom_start=15)
isochrones  = folium.features.GeoJson(isochrones_gdf.to_json())
osm_map.add_child(isochrones)
osm_map